<a href="https://colab.research.google.com/github/FabioMMaia/LLMs/blob/main/BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install gensim

In [ ]:
# !pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

array([0.        , 0.93729472, 0.        ])

In [ ]:
bm25.get_top_n(tokenized_query, corpus, n=1)

['It is quite windy in London']

In [16]:
pip install rank-bm25 langchain PyMuPDF

In [20]:
pip install --upgrade langchain

In [18]:
from google.colab import drive
import os
drive.mount('/content/drive')
wd = '/content/drive/MyDrive/Mestrado/POLI/BM25'
os.chdir(wd)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
pip install langchain-community langchain-core openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [25]:
import os
import fitz  # PyMuPDF
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import nltk
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate, LLMChain, OpenAI

# Certifique-se de que o NLTK está instalado
nltk.download('punkt')

def extrair_texto_do_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Função para dividir o texto usando RecursiveCharacterTextSplitter
def dividir_texto(text):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Ajuste o tamanho do chunk conforme necessário
        chunk_overlap=50  # Ajuste a sobreposição conforme necessário
    )
    return splitter.split_text(text)

# Caminho para a pasta que contém os PDFs
pasta_pdfs = "./pdfs"

# Listar todos os arquivos PDF na pasta
arquivos_pdfs = [f for f in os.listdir(pasta_pdfs) if f.endswith('.pdf')]

# Extrair e dividir texto dos PDFs preservando os rótulos
documentos = []
for arquivo_pdf in arquivos_pdfs:
    caminho_pdf = os.path.join(pasta_pdfs, arquivo_pdf)
    pdf_texto = extrair_texto_do_pdf(caminho_pdf)
    chunks = dividir_texto(pdf_texto)
    for chunk in chunks:
        documentos.append({"texto": chunk, "label": arquivo_pdf})

# Tokenizar o corpus
documentos_tokenizados = [word_tokenize(doc["texto"].lower()) for doc in documentos]

# Inicializar o BM25
bm25 = BM25Okapi(documentos_tokenizados)

# Definir a consulta
consulta = "Qual prazo do plano de ação?"
consulta_tokenizada = word_tokenize(consulta.lower())

# Obter scores do BM25
scores = bm25.get_scores(consulta_tokenizada)

# Ordenar os documentos por score
documentos_classificados = sorted(zip(scores, documentos), reverse=True, key=lambda x: x[0])

# Selecionar os top N documentos
top_n = 3
top_documentos = [doc for _, doc in documentos_classificados[:top_n]]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
top_documentos

[{'texto': '§ 2o  No requerimento enviado à SUSEP deverá constar o plano de ação e o \nprazo de implementação de cada item de deficiência. \n  \n§ 3o  Os prazos para implementação dos planos de ação serão contados a partir \nda data de protocolo do requerimento na SUSEP. \n \n§ 4o  Os prazos para saneamento das deficiências poderão ser diferenciados, em \nfunção da complexidade para resolução das mesmas. \n  \nArt. 3o  Os prazos para saneamento das deficiências não deverão ser maiores que \n90 (noventa) dias e, na hipótese de requerimento com  prazo superior, a sociedade/entidade \ndeverá justificar detalhadamente os motivos que impedem a solução da deficiência em \nprazo inferior, ficando  a critério do Conselho Diretor  da SUSEP o deferimento do pleito. \n \nArt. 4o  Caso a sociedade/entidade apresente, em conjunto com o requerimento \nde que trata o art. 2o, contestação quanto a 1 (uma) ou mais deficiências encontradas, e a',
  'label': '22027_6744.pdf'},
 {'texto': 'SUSEP a(s) cons

In [27]:
# Integrar com LangChain
template_prompt = """
Com base nos seguintes documentos:
{documentos}

Responda à seguinte pergunta:
{pergunta}
"""

prompt = PromptTemplate(input_variables=["documentos", "pergunta"], template=template_prompt)

In [29]:
with open("/content/drive/MyDrive/Projeto ML/2024/LLMs_RAGs_estudo/openai_key.txt", "r") as f:
  openai_key = f.read()

os.environ['OPENAI_API_KEY'] = openai_key

In [34]:
llm = OpenAI(temperature=0.7)

chain = LLMChain(llm=llm, prompt=prompt)

# Construir o input para o LangChain
documentos_str = "\n".join([f"Do {doc['label']}:\n{doc['texto']}" for doc in top_documentos])
pergunta = consulta

input_data = {"documentos": documentos_str, "pergunta": pergunta}

# Executar o LangChain
resposta = chain(input_data)
print(resposta)

{'documentos': 'Do 22027_6744.pdf:\n§ 2o  No requerimento enviado à SUSEP deverá constar o plano de ação e o \nprazo de implementação de cada item de deficiência. \n  \n§ 3o  Os prazos para implementação dos planos de ação serão contados a partir \nda data de protocolo do requerimento na SUSEP. \n \n§ 4o  Os prazos para saneamento das deficiências poderão ser diferenciados, em \nfunção da complexidade para resolução das mesmas. \n  \nArt. 3o  Os prazos para saneamento das deficiências não deverão ser maiores que \n90 (noventa) dias e, na hipótese de requerimento com  prazo superior, a sociedade/entidade \ndeverá justificar detalhadamente os motivos que impedem a solução da deficiência em \nprazo inferior, ficando  a critério do Conselho Diretor  da SUSEP o deferimento do pleito. \n \nArt. 4o  Caso a sociedade/entidade apresente, em conjunto com o requerimento \nde que trata o art. 2o, contestação quanto a 1 (uma) ou mais deficiências encontradas, e a\nDo 22027_6744.pdf:\nSUSEP a(s) con

In [37]:
resposta['text']

'\nO prazo do plano de ação é de 90 (noventa) dias, contados a partir da data de protocolo do requerimento na SUSEP, conforme estabelecido no § 3o do Art. 2o do documento Do 22027_6744.pdf.'

In [38]:
print(f"Prompt enviado ao LangChain:\n\n{template_prompt.format(documentos=documentos_str, pergunta=pergunta)}")


Prompt enviado ao LangChain:


Com base nos seguintes documentos:
Do 22027_6744.pdf:
§ 2o  No requerimento enviado à SUSEP deverá constar o plano de ação e o 
prazo de implementação de cada item de deficiência. 
  
§ 3o  Os prazos para implementação dos planos de ação serão contados a partir 
da data de protocolo do requerimento na SUSEP. 
 
§ 4o  Os prazos para saneamento das deficiências poderão ser diferenciados, em 
função da complexidade para resolução das mesmas. 
  
Art. 3o  Os prazos para saneamento das deficiências não deverão ser maiores que 
90 (noventa) dias e, na hipótese de requerimento com  prazo superior, a sociedade/entidade 
deverá justificar detalhadamente os motivos que impedem a solução da deficiência em 
prazo inferior, ficando  a critério do Conselho Diretor  da SUSEP o deferimento do pleito. 
 
Art. 4o  Caso a sociedade/entidade apresente, em conjunto com o requerimento 
de que trata o art. 2o, contestação quanto a 1 (uma) ou mais deficiências encontradas, e a
D